In [26]:
import cv2
import pandas as pd
import os
from torch.utils import data

In [2]:
DATASET_FOLDER = '/Users/DrMatters/Documents/git/vision/data/datasets/market1501/Market-1501-v15.09.15/'

In [16]:
import os
from typing import FrozenSet

import albumentations as albu
import pandas as pd
from torch.utils import data


class MarketDataset(data.Dataset):
    REQUIRED_COLUMNS: FrozenSet[str] = frozenset({'filename', 'pers_id'})

    def __init__(self, folder: str, index_df: pd.DataFrame = None,
                 transforms=albu.Compose([albu.HorizontalFlip()]),
                 preprocessing=None):
        assert set(index_df.columns).issubset(self.REQUIRED_COLUMNS), \
            ('Required columns are not present in dataframe. Expected:'
             f' {self.REQUIRED_COLUMNS}. Got: {set(index_df.columns)}')
        self.folder = folder
        self.index_df = index_df
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __getitem__(self, idx):
        row = self.index_df.loc[idx, :]
        filename = row['filename']
        person_id = row['pers_id']

        full_path = os.path.join(self.folder, filename)

        img = cv2.imread(full_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        augmented = self.transforms(image=img)
        img = augmented['image']
        if self.preprocessing:
            preprocessed = self.preprocessing(image=img)
            img = preprocessed['image']
        return img, person_id

    def __len__(self):
        return len(self.index_df)


In [17]:
def get_filenames_in_current(folder: str):
    walk = os.walk(folder)
    for current_catalog, sub_catalogs, files in walk:
        if current_catalog == folder:
            return sorted(files)

def create_index_df(folder: str):
    filenames = pd.Series(get_filenames_in_current(folder))
    filenames = filenames[filenames.str.endswith('jpg')]
    index_df = filenames.str.split('_', expand=True, n=2)
    index_df = index_df.iloc[:, :1]
    index_df = index_df.rename(columns={
        0: 'pers_id', 1: 'env_descr',
        2: 'orig_id'
    })
    index_df['filename'] = filenames

    return index_df

In [18]:
index_df = create_index_df(DATASET_FOLDER + 'bounding_box_test')
index_df.head()

,pers_id,filename
0,-1,-1_c1s1_000401_03.jpg
1,-1,-1_c1s1_000451_04.jpg
2,-1,-1_c1s1_001351_04.jpg
3,-1,-1_c1s1_001376_05.jpg
4,-1,-1_c1s1_011251_02.jpg


In [41]:
ds = MarketDataset(DATASET_FOLDER + 'bounding_box_test', index_df)

In [42]:
ld = data.DataLoader(ds, drop_last=True)

In [50]:
next(enumerate(ld))[1][0].numpy().squeeze().shape

(128, 64, 3)